The news articles dataset has been obtained from Kaggle. 
This is the url: https://www.kaggle.com/pariza/bbc-news-summary

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
import os
import gensim

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim

In [3]:
nlp = spacy.load('en_core_web_lg')

At the time of doing this project (25/12/2018), spacy's english language model (en_core_web_lg) has problem in recognizing stop words. This can be seen in the following example.

In [4]:
doc = nlp('This is a sentence. And the cat jumped over the dog. The cat returned as the prisoner of Azkaban.')
for token in doc:
    print(token.text, token.is_stop)

This False
is False
a False
sentence False
. False
And False
the False
cat False
jumped False
over False
the False
dog False
. False
The False
cat False
returned False
as False
the False
prisoner False
of False
Azkaban False
. False


As you can see above, it doesn't recognize 'This', 'is', 'a', 'the', etc. as stop words, we are going to address this issue, by the following steps.

In [5]:
for stop_word in STOP_WORDS:
    for word in (stop_word, stop_word.capitalize(), stop_word.upper()):
        lex = nlp.vocab[word]
        lex.is_stop = True

Let us recheck the above scenario again and see whether the model is recognizing stop words or not.

In [6]:
doc = nlp('This is a sentence. And the cat jumped over the dog. The cat returned as the prisoner of Azkaban.')
for token in doc:
    print(token.text, token.is_stop)

This True
is True
a True
sentence False
. False
And True
the True
cat False
jumped False
over True
the True
dog False
. False
The True
cat False
returned False
as True
the True
prisoner False
of True
Azkaban False
. False


We are going to analyze the news articles. Let us look at a sample news article.

In [7]:
with open('BBC News Summary/News Articles/politics/001.txt', 'r') as news_file:
    newsArticle = news_file.read()
    newsArticle = newsArticle.replace('\n', ' ')
    print(newsArticle)

Labour plans maternity pay rise  Maternity pay for new mothers is to rise by Â£1,400 as part of new proposals announced by the Trade and Industry Secretary Patricia Hewitt.  It would mean paid leave would be increased to nine months by 2007, Ms Hewitt told GMTV's Sunday programme. Other plans include letting maternity pay be given to fathers and extending rights to parents of older children. The Tories dismissed the maternity pay plan as "desperate", while the Liberal Democrats said it was misdirected.  Ms Hewitt said: "We have already doubled the length of maternity pay, it was 13 weeks when we were elected, we have already taken it up to 26 weeks. "We are going to extend the pay to nine months by 2007 and the aim is to get it right up to the full 12 months by the end of the next Parliament." She said new mothers were already entitled to 12 months leave, but that many women could not take it as only six of those months were paid. "We have made a firm commitment. We will definitely ext

We are going to start with the named entity recognition (NER) of the text document. NER is helpful in identifying geographical location, name of persons, organization, etc. in a textual document. Spacy's list of named entities can be foun here: https://spacy.io/api/annotation#named-entities

In [8]:
newsDoc = nlp(newsArticle)

In [9]:
for ent in newsDoc.ents:
    print(ent.text, ent.label_)

Â£1,400 ORG
the Trade and Industry ORG
Patricia Hewitt PERSON
nine months by 2007 DATE
Ms Hewitt PERSON
GMTV ORG
Sunday DATE
Tories NORP
Liberal NORP
Democrats NORP
Ms Hewitt PERSON
13 weeks DATE
up to 26 weeks DATE
nine months by 2007 DATE
the full 12 months DATE
the end of the next Parliament DATE
12 months DATE
months DATE
the six months DATE
is to nine months DATE
Â£1,400 PERSON
State ORG
Family PRODUCT
Theresa ORG
May DATE
Gordon Brown PERSON
December DATE
Tony Blair PERSON
Conservatives ORG
Democrat NORP
Sandra Gidley PERSON
the Liberal Democrats ORG
the first six months DATE
Ms Hewitt PERSON
David Frost PERSON
the British Chambers of Commerce ORG
Monday DATE
90% PERCENT
the first six weeks DATE
Â£102.80 a week DATE
six months old DATE


we can visualize the entities in the document using displacy

In [10]:
displacy.render(newsDoc, style='ent', jupyter=True)

Spacy is able to recognize most of the entities correctly, but as we can see above there a few places it is incorrect

For example, if we want to redact the name of persons from classified documents, manually doing it would be time consuming and requirement of lot of human resources. We can solve this by building a robust NER system, which can redact documents containing sensitive information. Let us do this in an example here. Let us take the above news article and redact all the names of person from it.

In [11]:
def getRedactedDoc(doc):

    tokens = []
    for ent in doc.ents:
        ent.merge()
        
    for token in doc:
        if token.ent_type_ == "PERSON":
            tokens.append("[REDACTED]")
        else:
            tokens.append(token.string)
            
    return "".join(tokens)

In [12]:
getRedactedDoc(newsDoc)

'Labour plans maternity pay rise  Maternity pay for new mothers is to rise by Â£1,400 as part of new proposals announced by the Trade and Industry Secretary [REDACTED].  It would mean paid leave would be increased to nine months by 2007, [REDACTED]told GMTV\'s Sunday programme. Other plans include letting maternity pay be given to fathers and extending rights to parents of older children. The Tories dismissed the maternity pay plan as "desperate", while the Liberal Democrats said it was misdirected.  [REDACTED]said: "We have already doubled the length of maternity pay, it was 13 weeks when we were elected, we have already taken it up to 26 weeks. "We are going to extend the pay to nine months by 2007 and the aim is to get it right up to the full 12 months by the end of the next Parliament." She said new mothers were already entitled to 12 months leave, but that many women could not take it as only six of those months were paid. "We have made a firm commitment. We will definitely extend

The news articles are grouped into 5 categories: sport, tech, politics, entertainment and business. Let's print a sample article from each category.

In [13]:
news_categories = ['sport', 'tech', 'politics', 'entertainment', 'business']
base_directory = 'BBC News Summary/News Articles/'

In [14]:
for category in news_categories:
    with open(base_directory + category + '/001.txt', 'r') as news_file:
        newsArticle = news_file.read()
        #we are going to print the first 400 characters of the article
        print('Category', category.capitalize())
        print('\n')
        if len(newsArticle) > 400:
            print(newsArticle[0:400] + '...')
        else:
            print(newsArticle)
        print('\n')

Category Sport


Claxton hunting first major medal

British hurdler Sarah Claxton is confident she can win her first major medal at next month's European Indoor Championships in Madrid.

The 25-year-old has already smashed the British record over 60m hurdles twice this season, setting a new mark of 7.96 seconds to win the AAAs title. "I am quite confident," said Claxton. "But I take each race as it comes. "As long...


Category Tech


Ink helps drive democracy in Asia

The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting.

This new technology is causing both worries and guarded optimism among different sectors of the population. In an effort to live up to its reputation in the 1990s a...


Category Politics


Labour plans maternity pay rise

Maternity pay for new mothers is to rise by Â£1,400 as part of new proposals announced by the Trade an

Let us count the number of news articles in each category.

In [15]:
for category in news_categories:
    num_articles = len([file for file in os.listdir(base_directory + category) if file.endswith(".txt")])
    print('Category:', category.capitalize(), ',' ,'Number of Articles:', num_articles)

Category: Sport , Number of Articles: 511
Category: Tech , Number of Articles: 401
Category: Politics , Number of Articles: 417
Category: Entertainment , Number of Articles: 386
Category: Business , Number of Articles: 510


We will be doing the topic modelling of the new articles. We will be using the gensim package for topic modelling. The different algorithms that we will be looking at:
1. Hierarchical Dirichlet Process (HDP)
2. Latent Dirichlet Allocation (LDA)
3. Latent Semantic Indexing (LSI)

In the case of news articles that are quite frequent, and can be put in the stopwords of the sapcy package

In [16]:
custom_stop_words = ['say', 'says', 'said', 'saying', '\'s', 'mr', 'ms', 'people']
for stop_word in custom_stop_words:
    for word in (stop_word, stop_word.capitalize()):
        lex = nlp.vocab[word]
        lex.is_stop = True

We will be using 200 articles from each news category for topic modelling

In [17]:
news_corpus = []

for category in news_categories:
    
    en_directory = os.fsencode(base_directory + category)
    count_articles = 0

    for file in os.listdir(en_directory):

        fileName = os.fsdecode(file)

        if fileName.endswith(".txt"):
            
            count_articles += 1
            
            if count_articles > 200:
                break
                
            cur_article = []

            with open(base_directory + category + '/' + fileName, 'r') as news_file:

                newsArticle = news_file.read()
                newsArticle = newsArticle.replace('\n', ' ')
                doc = nlp(newsArticle)

                for token in doc:

                    if token.lemma_ != '-PRON-' and not token.is_space and not token.is_stop and not token.is_punct and not token.like_num and not token.like_email and not token.like_url:

                        # For topic modelling lemmatized version of the word is preferable for better results
                        cur_article.append(token.lemma_)

            news_corpus.append(cur_article)
       

In [18]:
print(len(news_corpus))

1000


In [19]:
for i in (2, 202, 402, 602, 802):
    print(news_corpus[i])
    print('\n')

['greene', 'set', 'sight', 'world', 'title', 'maurice', 'greene', 'aim', 'wipe', 'pain', 'lose', 'olympic', 'm', 'title', 'athens', 'win', 'fourth', 'world', 'championship', 'crown', 'summer', 'settle', 'bronze', 'greece', 'fellow', 'american', 'justin', 'gatlin', 'francis', 'obikwelu', 'portugal', 'hurt', 'look', 'medal', 'mistake', 'lose', 'thing', 'greene', 'race', 'birmingham', 'friday', 'go', 'happen', 'goal', 'be', 'go', 'win', 'world', 'greene', 'cross', 'line', 'second', 'gatlin', 'win', 'second', 'close', 'fast', 'sprint', 'time', 'greene', 'believe', 'lose', 'race', 'title', 'semi', 'final', 'semi', 'final', 'race', 'win', 'race', 'conserve', 'energy', 'francis', 'obikwelu', 'come', 'take', 'not', 'know', 'believe', 'lane', 'final', 'lane', 'not', 'feel', 'race', 'feel', 'like', 'run', 'believe', 'middle', 'race', 'able', 'react', 'come', 'ahead', 'greene', 'deny', 'olympic', 'gold', '4x100', 'm', 'man', 'relay', 'catch', 'britain', 'mark', 'lewis', 'francis', 'final', 'leg',

In [20]:
bigram = gensim.models.Phrases(news_corpus)

In [21]:
news_corpus = [bigram[cur_article] for cur_article in news_corpus]

In [23]:
print(news_corpus[202])

['microsoft', 'seek', 'spyware', 'trojan', 'microsoft', 'investigate', 'trojan', 'program', 'attempt', 'switch', 'firm', 'anti_spyware', 'software', 'spyware_tool', 'release', 'microsoft', 'week', 'download', 'stephen', 'toulouse', 'security', 'manager', 'microsoft', 'malicious_program', 'call', 'bankash', 'trojan', 'send', 'e_mail', 'attachment', 'microsoft', 'believe', 'program', 'widespread', 'recommend', 'user', 'use', 'anti_virus', 'program', 'program', 'attempt', 'disable', 'delete', 'microsoft', 'anti_spyware', 'tool', 'suppress', 'warning', 'message', 'give', 'user', 'try', 'steal', 'online', 'banking', 'password', 'personal_information', 'track', 'user', 'keystroke', 'microsoft', 'statement', 'investigate', 'call', 'criminal', 'attack', 'software', 'earlier_week', 'microsoft', 'buy', 'anti_virus', 'software', 'maker', 'sybari', 'software', 'improve', 'security', 'windows', 'e_mail', 'software', 'microsoft', 'plan', 'offer', 'pay', 'anti_virus', 'software', 'set', 'date', 'rele

In [24]:
dictionary = Dictionary(news_corpus)
print(dictionary)

Dictionary(17167 unique tokens: ['25-year_old', 'aaas_title', 'athlete', 'attention', 'bear']...)


In [25]:
corpus_for_modelling = [dictionary.doc2bow(cur_article) for cur_article in news_corpus]

In [26]:
print(corpus_for_modelling[2])

[(5, 1), (12, 5), (14, 1), (22, 1), (27, 1), (42, 1), (53, 6), (59, 1), (60, 3), (61, 2), (69, 1), (70, 2), (71, 4), (73, 1), (77, 4), (78, 5), (90, 1), (101, 1), (124, 1), (135, 1), (136, 1), (137, 1), (138, 1), (139, 1), (140, 1), (141, 1), (142, 1), (143, 1), (144, 1), (145, 1), (146, 4), (147, 1), (148, 1), (149, 1), (150, 1), (151, 1), (152, 1), (153, 1), (154, 1), (155, 1), (156, 2), (157, 1), (158, 1), (159, 1), (160, 1), (161, 1), (162, 1), (163, 1), (164, 1), (165, 2), (166, 1), (167, 1), (168, 3), (169, 2), (170, 3), (171, 1), (172, 1), (173, 1), (174, 8), (175, 1), (176, 1), (177, 1), (178, 1), (179, 1), (180, 1), (181, 1), (182, 1), (183, 1), (184, 1), (185, 2), (186, 1), (187, 1), (188, 1), (189, 3), (190, 1), (191, 1), (192, 1), (193, 1), (194, 1), (195, 1), (196, 1), (197, 1), (198, 1), (199, 1), (200, 2), (201, 1), (202, 1), (203, 1), (204, 1), (205, 1), (206, 1), (207, 1), (208, 1), (209, 1), (210, 1), (211, 1), (212, 1), (213, 2), (214, 1), (215, 1), (216, 1), (217, 1

We are going to start with the Hierarchical Dirichlet Process (HDP) Topic Modelling Algorithm. In HDP, we don't need to specify the number of topics. The algorithm is able o deduce the best "number of topics" for a given text corpus.

In [29]:
hdpModel = HdpModel(corpus=corpus_for_modelling, id2word=dictionary)

In [30]:
hdpModel.show_topics()

[(0,
  '0.006*year + 0.004*new + 0.003*government + 0.003*film + 0.003*company + 0.003*good + 0.003*time + 0.003*uk + 0.002*rise + 0.002*work + 0.002*firm + 0.002*economy + 0.002*report + 0.002*market + 0.002*include + 0.002*world + 0.002*country + 0.002*go + 0.002*number + 0.002*add'),
 (1,
  '0.004*year + 0.003*new + 0.002*government + 0.002*uk + 0.002*sale + 0.002*profit + 0.002*firm + 0.002*m + 0.002*$ + 0.002*rise + 0.002*good + 0.002*number + 0.002*film + 0.002*game + 0.002*come + 0.002*high + 0.001*report + 0.001*market + 0.001*plan + 0.001*play'),
 (2,
  '0.004*year + 0.003*new + 0.002*company + 0.002*rise + 0.002*government + 0.002*time + 0.002*plan + 0.002*hunt + 0.002*uk + 0.001*law + 0.001*boeing + 0.001*murder + 0.001*deal + 0.001*expect + 0.001*come + 0.001*profit + 0.001*report + 0.001*world + 0.001*police + 0.001*share'),
 (3,
  '0.004*year + 0.003*sale + 0.002*m + 0.002*new + 0.001*uk + 0.001*come + 0.001*liverpool + 0.001*week + 0.001*parry + 0.001*see + 0.001*company

In [31]:
hdp_coherence_model = CoherenceModel(model=hdpModel, texts=news_corpus, dictionary=dictionary, coherence='c_v')
hdp_coherence = hdp_coherence_model.get_coherence()
print(hdp_coherence)

0.5685086948425858


Next, we are going to proceed with the Latent Dirichlet Allocation model, here we need to pass the number of topics. Based on the coherence score, we can deduce the best "number of topics".

In [32]:
list_num_of_topics = [10, 15, 20, 25, 30]
for num_topics in list_num_of_topics:
    ldaModel = LdaModel(corpus=corpus_for_modelling, num_topics=num_topics, id2word=dictionary)
    coherenceModel = CoherenceModel(model=ldaModel, texts=news_corpus, dictionary=dictionary, coherence='c_v')
    print('Number of Topics', num_topics)
    print('Coherence Value', coherenceModel.get_coherence())

Number of Topics 10
Coherence Value 0.23663326206920732
Number of Topics 15
Coherence Value 0.23199814916643471
Number of Topics 20
Coherence Value 0.24730302524458345
Number of Topics 25
Coherence Value 0.23852485060215592
Number of Topics 30
Coherence Value 0.2514537514689832


In [33]:
ldaModel_best = LdaModel(corpus=corpus_for_modelling, num_topics=30, id2word=dictionary)
ldaModel_best.show_topics(30)

[(0,
  '0.005*"year" + 0.004*"play" + 0.004*"firm" + 0.003*"new" + 0.003*"home" + 0.003*"week" + 0.003*"add" + 0.003*"microsoft" + 0.003*"game" + 0.003*"go"'),
 (1,
  '0.007*"year" + 0.005*"new" + 0.004*"good" + 0.004*"come" + 0.004*"plan" + 0.004*"time" + 0.003*"government" + 0.003*"add" + 0.003*"world" + 0.003*"report"'),
 (2,
  '0.011*"year" + 0.005*"new" + 0.004*"number" + 0.003*"go" + 0.003*"time" + 0.003*"month" + 0.003*"come" + 0.003*"market" + 0.003*"law" + 0.003*"good"'),
 (3,
  '0.009*"m" + 0.007*"film" + 0.005*"year" + 0.003*"company" + 0.003*"government" + 0.003*"take" + 0.003*"time" + 0.003*"good" + 0.003*"japan" + 0.003*"win"'),
 (4,
  '0.005*"year" + 0.005*"good" + 0.004*"game" + 0.004*"bill" + 0.003*"use" + 0.003*"play" + 0.003*"add" + 0.003*"film" + 0.003*"come" + 0.003*"look"'),
 (5,
  '0.006*"year" + 0.005*"work" + 0.004*"new" + 0.004*"government" + 0.003*"film" + 0.003*"day" + 0.003*"net" + 0.003*"good" + 0.003*"e_mail" + 0.003*"country"'),
 (6,
  '0.009*"year" + 0.

In [34]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldaModel_best, corpus_for_modelling, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.024424  0.012753       1        1  7.767267
14     0.005745  0.004111       2        1  7.446269
26     0.009739  0.022189       3        1  6.587902
23     0.017175  0.014957       4        1  5.738687
11     0.032916  0.007717       5        1  4.692606
15    -0.050242  0.030740       6        1  4.436525
12     0.010496  0.028260       7        1  3.966869
13     0.018166 -0.003784       8        1  3.936393
5     -0.002924  0.036415       9        1  3.551637
27    -0.027447  0.046039      10        1  3.433524
17     0.010228 -0.012826      11        1  3.327549
9     -0.008056  0.008213      12        1  3.307653
21     0.031622 -0.033275      13        1  3.289021
29     0.023972 -0.022542      14        1  3.108044
8     -0.014169 -0.009200      15        1  3.075836
28    -0.001327  0.014013      16        1  2.956820
20     0.040947  0.017577      17        1  2.953796
22     0.029652 -0.000425      18        1  2.746611
16    -0.008143 -0.003681      19        1  2.522717
6     -0.016910 -0.035592      20        1  2.420146
1      0.006183  0.000848      21        1  2.375403
19    -0.015468 -0.021616      22        1  2.371290
25    -0.049775 -0.001360      23        1  2.098178
18    -0.035328 -0.059576      24        1  2.073044
24     0.009626  0.000368      25        1  1.975652
7     -0.044367 -0.000278      26        1  1.831040
0     -0.007743  0.014625      27        1  1.798923
3     -0.009461 -0.047566      28        1  1.585536
4      0.033131  0.006816      29        1  1.490309
10     0.036184 -0.013919      30        1  1.134745, topic_info=      Category         Freq        Term        Total  loglift  logprob
term                                                                  
5884   Default   416.000000        film   416.000000  30.0000  30.0000
190    Default   455.000000           m   455.000000  29.0000  29.0000
79     Default  1054.000000        year  1054.000000  28.0000  28.0000
1273   Default   446.000000        game   446.000000  27.0000  27.0000
77     Default   415.000000         win   415.000000  26.0000  26.0000
345    Default   566.000000        good   566.000000  25.0000  25.0000
2668   Default   185.000000        site   185.000000  24.0000  24.0000
796    Default   399.000000     company   399.000000  23.0000  23.0000
1314   Default   358.000000        play   358.000000  22.0000  22.0000
46     Default   703.000000         new   703.000000  21.0000  21.0000
219    Default   358.000000        take   358.000000  20.0000  20.0000
5381   Default   135.000000   microsoft   135.000000  19.0000  19.0000
15877  Default    49.000000        fiat    49.000000  18.0000  18.0000
78     Default   392.000000       world   392.000000  17.0000  17.0000
1785   Default   446.000000  government   446.000000  16.0000  16.0000
3168   Default    59.000000           v    59.000000  15.0000  15.0000
217    Default   195.000000        star   195.000000  14.0000  14.0000
5454   Default    66.000000          bt    66.000000  13.0000  13.0000
198    Default   347.000000         not   347.000000  12.0000  12.0000
3084   Default   271.000000        rise   271.000000  11.0000  11.0000
1159   Default   205.000000       award   205.000000  10.0000  10.0000
803    Default   337.000000        firm   337.000000   9.0000   9.0000
70     Default   572.000000        time   572.000000   8.0000   8.0000
4195   Default    89.000000       china    89.000000   7.0000   7.0000
5342   Default   178.000000       child   178.000000   6.0000   6.0000
279    Default   323.000000      report   323.000000   5.0000   5.0000
5971   Default   144.000000        bill   144.000000   4.0000   4.0000
1994   Default   267.000000      market   267.000000   3.0000   3.0000
76     Default   285.000000        week   285.000000   2.0000   2.0000
3132   Default   121.000000     chelsea   121.000000   1.000

Lastly, we are going to proceed with the Latent Semantic Indexing model, here we need to pass the number of topics. Based on the coherence score, we can deduce the best "number of topics".

In [35]:
for num_topics in list_num_of_topics:
    lsiModel = LsiModel(corpus=corpus_for_modelling, num_topics=num_topics, id2word=dictionary)
    coherenceModel = CoherenceModel(model=lsiModel, texts=news_corpus, dictionary=dictionary, coherence='c_v')
    print('Number of Topics', num_topics)
    print('Coherence Value', coherenceModel.get_coherence())

Number of Topics 10
Coherence Value 0.37252656289077757
Number of Topics 15
Coherence Value 0.37226546370652963
Number of Topics 20
Coherence Value 0.33798064817110934
Number of Topics 25
Coherence Value 0.3195759443226098
Number of Topics 30
Coherence Value 0.33123704828133577


In [36]:
lsiModel = LsiModel(corpus=corpus_for_modelling, num_topics=10, id2word=dictionary)
lsiModel.show_topics(10)

[(0,
  '0.251*"year" + 0.190*"new" + 0.160*"good" + 0.153*"time" + 0.144*"game" + 0.122*"government" + 0.114*"come" + 0.114*"uk" + 0.114*"m" + 0.107*"work"'),
 (1,
  '-0.337*"game" + -0.243*"m" + -0.229*"win" + 0.181*"government" + -0.162*"good" + -0.155*"film" + -0.151*"play" + -0.128*"time" + 0.118*"service" + -0.117*"title"'),
 (2,
  '-0.430*"game" + 0.210*"m" + -0.204*"technology" + 0.199*"government" + 0.150*"year" + -0.140*"mobile" + -0.126*"play" + -0.121*"user" + -0.117*"phone" + -0.111*"gadget"'),
 (3,
  '0.335*"year" + 0.240*"film" + -0.187*"liverpool" + -0.179*"not" + 0.166*"m" + 0.163*"sale" + -0.144*"parry" + -0.140*"government" + 0.139*"rise" + -0.135*"think"'),
 (4,
  '-0.434*"game" + 0.326*"film" + -0.157*"sale" + 0.156*"site" + -0.133*"rise" + 0.128*"m" + 0.122*"user" + -0.122*"market" + -0.109*"economy" + 0.106*"award"'),
 (5,
  '-0.311*"game" + -0.250*"government" + -0.237*"film" + 0.224*"liverpool" + 0.188*"parry" + 0.169*"m" + 0.146*"gerrard" + -0.134*"child" + 0.1